In [ ]:
# 12-18-2022
# benjamin m hardy
# Purpose: demonstrate relationships dictating relative noise contributions from 
# some key references:

# great intro reference is Noise in MRI - Al macovski
# [1] https://pubmed.ncbi.nlm.nih.gov/8875425/
# another more recent interesting take:
# [2] https://ieeexplore.ieee.org/document/5910383

# great description of Q-relationship:
# [3] https://pubmed.ncbi.nlm.nih.gov/10680678/
# would be good to improve on this paper's assumptions and to show it experimentally.

# Overall great reference is Darasse and Ginefri
# [4] https://pubmed.ncbi.nlm.nih.gov/14652180/

# key question: How does temperature, coil size, conductivity of sample, influence noise contributions
# figure 4 in Darasse and Ginefri is the experimental demonstration we would want to show.


In [ ]:
# imports:
from math import pi
import numpy
import matplotlib.pyplot as plt


In [ ]:
# figure 1: 
# sample resistance and coil resistance vs coil size
# for a uniform sphere (eq. 6 in ref [4], also hoult and Richards)
#
b = .005 # sample radius
sigma = .66 # conductivity of sample, S/m
omega = 2*pi*650e6 # resonance frequency, 650 MHz
cc = 1 # coupling coefficient (B1/Ic) B1 over the current in coil
rs = 2*pi*sigma*omega**2 *cc**2 * b**5
 

In [ ]:
# figure 2:
# temperature of coil and conductivity of sample